# Example of using Agent node

In [ ]:
import os
from dotenv import load_dotenv
from aic_flow.nodes.ai import Agent


load_dotenv()


model_config = {
    "model": "gpt-4o-mini",
    "api_key": os.getenv("OPENAI_API_KEY"),
}

## Structured output with vanilla JSON dict

In [2]:
so_schema = """
json_dict = {
    "type": "object",
    "title": "Person",
    "description": "A person",
    "properties": {
        "name": {"type": "string"},
    },
    "required": ["name"],
}
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'Jane Smith'}

## Structured output with OpenAI JSON dict

In [3]:
so_schema = """
oai_json_schema = {
    "name": "get_person",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {"name": {"type": "string"}},
        "additionalProperties": False,
        "required": ["name"],
    },
}
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}

## Structured output with Pydantic Models

In [4]:
so_schema = """
class Person(BaseModel):
    \"\"\"A Person.\"\"\"

    name: str
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

Person(name='John Doe')

## Structured output with Typed dict

In [5]:
so_schema = """
class Person(TypedDict):
    \"\"\"A Person.\"\"\"

    name: str
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}